In [1]:
# Check CUDA version
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
# Install CUDA package
!pip install git+https://github.com/afnan47/cuda.git


  Cloning https://github.com/afnan47/cuda.git to /tmp/pip-req-build-ty9zr79n
  Running command git clone --filter=blob:none --quiet https://github.com/afnan47/cuda.git /tmp/pip-req-build-ty9zr79n
  Resolved https://github.com/afnan47/cuda.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4289 sha256=44a48b64e57884a41828559a3b7b7f1814ffa2583898e520f9208883274076a8
  Stored in directory: /tmp/pip-ephem-wheel-cache-33werrxw/wheels/aa/f3/44/e10c1d226ec561d971fcd4b0463f6bff08602afa928a3e7bc7
Successfully built NVCCPlugin


In [3]:
# Load nvcc plugin
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [4]:
%%writefile sum.cu
#include <iostream>
#include <vector>
#include <climits>

__global__ void min_reduction_kernel(int* arr, int size, int* result) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < size) {
        atomicMin(result, arr[tid]);
    }
}

__global__ void max_reduction_kernel(int* arr, int size, int* result) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < size) {
        atomicMax(result, arr[tid]);
    }
}

__global__ void sum_reduction_kernel(int* arr, int size, int* result) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < size) {
        atomicAdd(result, arr[tid]);
    }
}

__global__ void average_reduction_kernel(int* arr, int size, int* sum) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < size) {
        atomicAdd(sum, arr[tid]);
    }
}

int main() {
    std::vector<int> arr = {5, 2, 9, 1, 7, 6, 8, 3, 4};
    int size = arr.size();
    int* d_arr;
    int* d_result;
    int result_min = INT_MAX;
    int result_max = INT_MIN;
    int result_sum = 0;

    // Allocate memory on the device
    cudaMalloc(&d_arr, size * sizeof(int));
    cudaMalloc(&d_result, sizeof(int));

    // Copy data from host to device
    cudaMemcpy(d_arr, arr.data(), size * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_result, &result_min, sizeof(int), cudaMemcpyHostToDevice);

    // Perform min reduction
    min_reduction_kernel<<<(size + 255) / 256, 256>>>(d_arr, size, d_result);
    cudaMemcpy(&result_min, d_result, sizeof(int), cudaMemcpyDeviceToHost);
    std::cout << "Minimum value: " << result_min << std::endl;

    // Perform max reduction
    cudaMemcpy(d_result, &result_max, sizeof(int), cudaMemcpyHostToDevice);
    max_reduction_kernel<<<(size + 255) / 256, 256>>>(d_arr, size, d_result);
    cudaMemcpy(&result_max, d_result, sizeof(int), cudaMemcpyDeviceToHost);
    std::cout << "Maximum value: " << result_max << std::endl;

    // Perform sum reduction
    cudaMemcpy(d_result, &result_sum, sizeof(int), cudaMemcpyHostToDevice);
    sum_reduction_kernel<<<(size + 255) / 256, 256>>>(d_arr, size, d_result);
    cudaMemcpy(&result_sum, d_result, sizeof(int), cudaMemcpyDeviceToHost);
    std::cout << "Sum: " << result_sum << std::endl;

    // Perform average reduction
    cudaMemcpy(d_result, &result_sum, sizeof(int), cudaMemcpyHostToDevice);
    average_reduction_kernel<<<(size + 255) / 256, 256>>>(d_arr, size, d_result);
    cudaMemcpy(&result_sum, d_result, sizeof(int), cudaMemcpyDeviceToHost);
    std::cout << "Average: " << static_cast<double>(result_sum) / size << std::endl;

    // Free device memory
    cudaFree(d_arr);
    cudaFree(d_result);

    return 0;
}

Writing sum.cu


In [5]:
!nvcc sum.cu -o sum
!./sum

Minimum value: 1
Maximum value: 9
Sum: 45
Average: 10


In [6]:
%%writefile add.cu


#include <iostream>

__global__ void arradd(int *x, int *y, int *z)
{
    int id = blockIdx.x;
    z[id] = x[id] + y[id];
}

int main()
{
    const int size = 6;
    int a[size], b[size], c[size];

    std::cout << "Enter six elements of first array:\n";
    for (int i = 0; i < size; i++)
    {
        std::cin >> a[i];
    }

    std::cout << "Enter six elements of second array:\n";
    for (int i = 0; i < size; i++)
    {
        std::cin >> b[i];
    }

    int *d, *e, *f;

    cudaMalloc((void **)&d, size * sizeof(int));
    cudaMalloc((void **)&e, size * sizeof(int));
    cudaMalloc((void **)&f, size * sizeof(int));

    cudaMemcpy(d, a, size * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(e, b, size * sizeof(int), cudaMemcpyHostToDevice);

    arradd<<<size, 1>>>(d, e, f);

    cudaMemcpy(c, f, size * sizeof(int), cudaMemcpyDeviceToHost);

    std::cout << "Sum of two arrays:\n";
    for (int i = 0; i < size; i++)
    {
        std::cout << c[i] << "\t";
    }
    std::cout << std::endl;

    cudaFree(d);
    cudaFree(e);
    cudaFree(f);

    return 0;
}


Writing add.cu


In [7]:
!nvcc add.cu -o add
!./add

Enter six elements of first array:
1
2
3
4
5
6
Enter six elements of second array:
8
7
5
4
3
1
Sum of two arrays:
9	9	8	8	8	7	
